# Кредит
**Источник:** <br>https://www.kaggle.com/
<br>
<br>
**Цели:** <br>● провести предобработку данных, для последующего их удобного анализа
<br>
● проанализировать данные в соответсвии с имеющимися задачами
<br>
<br>
**Задачи:** <br>● Выяснить как количество детей у клиента влияет на погашение кредита без задолженностей
<br>
● Выяснить как цель получения кредита влияет на погашение кредита?
<br>
● Выяснить как семейное положение влияет на погашение кредита без задолженностей
<br>
<br>
**Входные данные:** <br>входные данные представляют собой файл csv, содержащий в себе информацию о кредиторах и их некоторых параметрах
<br>
<br>
**Используемые библиотеки:** <br>pandas
<br>numpy
<br>re
<br>
<br>
Для начала импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import re

### Обзор данных

Считаем имеющийся файл и запишем его в переменную df.
<br>
Предварительно открыв его через обычный текстовый редактор на компьютере, стало понятно, что кодировка нормальная и разделение соответсвует общепринятому (;)

In [2]:
df = pd.read_csv("summative_1.csv")

Выведем основную информацию.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   Children          21525 non-null  int64  
 2   Age               21525 non-null  int64  
 3   Education         21525 non-null  object 
 4   FamilyStatus      21525 non-null  object 
 5   Debt              21525 non-null  int64  
 6   MonthlyIncome     19351 non-null  float64
 7   PurposeOfTheLoan  21525 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 1.3+ MB


**Описание данных:**<br>
● Children — число детей
<br>
● Age — возраст
<br>
● Education — уровень образования
<br>
● FamilyStatus — семейное положение
<br>
● Debt — была ли задолженность по возврату кредита
<br>
● MonthlyIncome — ежемесячный доход
<br>
● PurposeOfTheLoan — цель получения кредита
<br>
<br>

Мы имеем 21525 строк и 8 колонок. Также в некоторых строчках имеются пропуски. Выведем таблицу и посмотрим, что хранится в каждом из столбцов.


In [4]:
df.head()

,Unnamed: 0,Children,Age,Education,FamilyStatus,Debt,MonthlyIncome,PurposeOfTheLoan
0,0,1,42,высшее,женат / замужем,0,253875.639453,покупка жилья
1,1,1,36,среднее,женат / замужем,0,112080.014102,приобретение автомобиля
2,2,0,33,Среднее,женат / замужем,0,145885.952297,покупка жилья
3,3,3,32,среднее,женат / замужем,0,267628.550329,дополнительное образование
4,4,0,53,среднее,гражданский брак,0,158616.077870,сыграть свадьбу


Заметим, что столбцы имеют правильный тип (он соответсвует данным, которые хранятся в том или ином столбце). То есть:
<br>
Children - int
<br>
Age - int
<br>
Education - object
<br>
Debt - int
<br>
MonthlyIncome - float
<br>
PurposeOfTheLoan - int
<br>
<br>
### Микро-вывод: обзор
Мы имеем данные по кредитованию, которые нам в дальнейшем надо будет проанализировать. Данных достаточно много (21525 строк), что позволяет говорить о релевантности анализа. В ходе обзора каждый столбец был описан, а также проверен его тип. В процессе предобработки предстоит избавиться от пропусков, которые видно в основной информации о данных. 
## Предобработка данных
### Названия столбцов и лишние столбцы
Названия столбцов не соотвествуют стилю **snake_case**, исправим это:

In [5]:
df.columns = df.columns.str.replace(r'(?<!^)(?=[A-Z])', "_").str.lower()

/var/folders/rt/mvnmln8j70d9fmnh5b6pl1qr0000gn/T/ipykernel_11720/1081600409.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r'(?<!^)(?=[A-Z])', "_").str.lower()


In [6]:
df.columns

Index(['unnamed: 0', 'children', 'age', 'education', 'family_status', 'debt',
       'monthly_income', 'purpose_of_the_loan'],
      dtype='object')

Теперь все верно. Поймем, что столбик **"unnamed:_0"**_ является просто дубликатом индекса, так что его можно удалить:

In [7]:
df = df.drop(["unnamed: 0"], axis=1)

### Пропуски

Проверим наличие пропусков и проанализируем их процент в каждом из столбиков:

In [8]:
df.isna().mean()

children               0.000000
age                    0.000000
education              0.000000
family_status          0.000000
debt                   0.000000
monthly_income         0.100999
purpose_of_the_loan    0.000000
dtype: float64

Заметим, что пропуски есть только в столбце **"monthly_income"**, причем их там 10%, что достаточно много, чтобы просто удалять строки. С другой стороны не понятно, есть ли вообще доход у людей, у которых он не указан. Если посмотреть по данным, то разброс дохода, даже если смотреть людей примерно одного возраста и имеющих одинаковое образование, очень большой. Запомним, что с этим надо разобраться, но перед тем как присутпить к обработке пропусков, заменим все неявные дубликаты в остальных столбцах:
### Обработка неявнях дубликатов

In [9]:
df["education"].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Неявных дубликатов много, приведем все варианты в нижний регистр:

In [10]:
df.loc[:, 'education'] = df.loc[:, 'education'].str.lower()
df["education"].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Отлично, теперь для собственного удобства заменим их следующим образом:
<br>
среднее - ср
<br>
высшее - вы
<br>
неоконченное высшее - не
<br>
начальное - на
<br>
ученая степень - уч
<br>

In [11]:

df['education'] = df["education"].apply(lambda x: x[0:2])
df.head(20)

,children,age,education,family_status,debt,monthly_income,purpose_of_the_loan
0,1,42,вы,женат / замужем,0,253875.639453,покупка жилья
1,1,36,ср,женат / замужем,0,112080.014102,приобретение автомобиля
2,0,33,ср,женат / замужем,0,145885.952297,покупка жилья
3,3,32,ср,женат / замужем,0,267628.550329,дополнительное образование
4,0,53,ср,гражданский брак,0,158616.077870,сыграть свадьбу
5,0,27,вы,гражданский брак,0,255763.565419,покупка жилья
6,0,43,вы,женат / замужем,0,240525.971920,операции с жильем
7,0,50,ср,женат / замужем,0,135823.934197,образование
8,2,35,вы,гражданский брак,0,95856.832424,на проведение свадьбы
9,0,41,ср,женат / замужем,0,144425.938277,покупка жилья для семьи


Проверим неявные дубликаты в столбце **"family_status"**:

In [12]:
df["family_status"].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Неявеных дубликатов нет, но для убоства также заменим:
<br>
женат / замужем - ж/з
<br>
гражданский брак - гб
<br>
Не женат / не замужем - нет
<br>
в разводе - р
<br>
вдовец / вдова - вд
<br>

In [13]:
df.loc[:, 'family_status'] = df.loc[:, 'family_status'].replace("женат / замужем", "ж/з")
df.loc[:, 'family_status'] = df.loc[:, 'family_status'].replace("гражданский брак", "гб")
df.loc[:, 'family_status'] = df.loc[:, 'family_status'].replace("Не женат / не замужем", "нет")
df.loc[:, 'family_status'] = df.loc[:, 'family_status'].replace("в разводе", "р")
df.loc[:, 'family_status'] = df.loc[:, 'family_status'].replace("вдовец / вдова", "вд")

Здесь тоже можно было бы написать функцию, но обрабатывать все случаи if-ами было бы неудобнее.
### Обработка явных дубликатов

Теперь проверим на явные дубликаты и в случае обнаружения удалим их:

In [14]:
df.duplicated().sum()

192

Это я делаю после обработки неявных дубликатов во всех столбцах, кроме **"purpose_of_the_loan"**, т.к. его я буду обрабатывать отдельно, создавая словарь значений, чтобы избежать удаления строк, изначально не являющихся дубликатами, но похожих на них. Удалим:

In [15]:
df = df.drop_duplicates()

### И снова к неявным дубликатам
Посмотрим на неявные дубликаты колонки **"purpose_of_the_loan"**

In [16]:
df["purpose_of_the_loan"].value_counts()

свадьба                                   784
сыграть свадьбу                           758
на проведение свадьбы                     758
операции с недвижимостью                  674
покупка коммерческой недвижимости         657
операции с коммерческой недвижимостью     649
операции с жильем                         648
покупка жилья для сдачи                   647
покупка жилья                             645
жилье                                     642
покупка жилья для семьи                   632
строительство собственной недвижимости    632
недвижимость                              626
операции со своей недвижимостью           625
строительство жилой недвижимости          623
покупка своего жилья                      616
строительство недвижимости                615
покупка недвижимости                      611
покупка жилой недвижимости                603
ремонт жилью                              603
на покупку своего автомобиля              504
заняться высшим образованием      

Заметим, что все значения можно разделить на некоторые категории, которые помогут обобщить, но при этом причина взятие кредита не поменяется:
<br>
"свадьба"
<br>
"ремонт"
<br>
"образование"
<br>
"автомобиль"
<br>
"жилье"
<br>
"строительство"
<br>
"недвижимость"
Создадим функцию и заменим значения на соответствуюшие: 

In [17]:
d = dict()
def checking(x: str) -> str:
    """
    функция принимает на вход значение ячейки столбца 'purpose_of_the_loan' и проверяет, 
    является ли оно ключом в словаре d. Если да, то возвращается значение по этому ключу.
    Если нет, то вызывается функция 'changing', добавляется новый ключ в словарь и возвращается 
    значение, полученное функцией 'changing'
    
    """
    if x in d:
        return d[x]
    else:
        a = changing(x) 
        d[x]= a
        return a
        
def changing(x: str) -> str:
    """
    функция принимает на вход значение ячейки столбца 'purpose_of_the_loan' и смотрит сколько букв
    совпадает со всеми из ключевых слов и в соответсвии с выполнением какого-либо из условий совпадения 
    нужного количества букв возвращает замену
    
    """
    common_weeding = set(x) & set("свадьб")
    common_repair = set(x) & set("ремонт")
    common_build = set(x) & set("строительство")
    common_car = set(x) & set("автомобил")
    common_study = set(x) & set("образов")
    common_kom = set(x) & set("комерч")
    common_house = set(x) & set("жил")
    if len(common_weeding) == 6:
        return "свадьба"
    elif len(common_build) == 9:
        return "строительство"
    elif len(common_kom) == 6:
        return "недвижимость"
    elif len(common_car) == 8:
        return "автомобиль"
    elif len(common_repair) == 6:
        return "ремонт"
    elif len(common_study) == 6:
        return "образование"
    elif len(common_house) == 3:
        return "жилье"
    else:
        return "недвижимость"
    


df["purpose_of_the_loan"] = df["purpose_of_the_loan"].apply(checking)
df.head(10)

,children,age,education,family_status,debt,monthly_income,purpose_of_the_loan
0,1,42,вы,ж/з,0,253875.639453,жилье
1,1,36,ср,ж/з,0,112080.014102,автомобиль
2,0,33,ср,ж/з,0,145885.952297,жилье
3,3,32,ср,ж/з,0,267628.550329,образование
4,0,53,ср,гб,0,158616.077870,свадьба
5,0,27,вы,гб,0,255763.565419,жилье
6,0,43,вы,ж/з,0,240525.971920,жилье
7,0,50,ср,ж/з,0,135823.934197,образование
8,2,35,вы,гб,0,95856.832424,свадьба
9,0,41,ср,ж/з,0,144425.938277,жилье


### Поиск анамальных значений
Проверим остальные столбцы на "адекватность значений" (чтобы у нас не было человека, которому 500 лет и зарабатывает он -10 млрд). Проверим столбец **"children"**

In [18]:
df["children"].value_counts()

 0     13995
 1      4791
 2      2044
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-1 ребенок и 20 детей - те данные, которые стоит удалить. Конечно етсь вероятность, что у кого-то будет 20 детей, но это настолько редкий случай, что вероятно это опечатка. (согласно статистике: https://journal.tinkoff.ru/one-two-three-stat/?ysclid=l8k7tjdmmj190692755 семьи, где больше 5ти детей - уже большая редкость)
<br>
Так как этих значений не много, можно просто их удалить

In [19]:
df = df.query("children not in [-1, 20]"

Проверим столбик **"age"**

In [20]:
df["age"].value_counts()

35    609
41    600
40    598
34    594
38    589
42    587
33    576
39    567
31    553
36    552
44    541
29    541
48    531
30    529
37    528
43    506
50    503
32    503
49    501
28    501
27    490
45    489
52    479
56    474
47    469
54    468
46    465
53    455
57    449
58    447
51    440
59    439
55    432
26    405
60    369
25    356
61    352
62    342
63    267
24    263
64    255
23    249
65    192
22    183
66    181
67    166
21    109
0     100
68     99
69     83
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: age, dtype: int64

Значение 0 - не очень хорошо. Остальные значения подходят, так как кредит могут выдать любому человеку, достигшему совершеннолетия. Строки со значением "0" стоит удалить:

In [21]:
df = df.query("age not in [0]")

Проверим столбец **"debt"**

In [22]:
df["debt"].value_counts()

0    19387
1     1723
Name: debt, dtype: int64

Тут все хорошо, ничего удалять не нужно. Проверим, что зарплаты лежат в нормальном диапазоне:

In [23]:
df["monthly_income"].value_counts(ascending=True)

253875.639453    1
157205.834562    1
114842.854099    1
282354.554215    1
235961.235873    1
                ..
172637.167134    1
142980.588554    1
157732.709448    1
88361.884631     1
82047.418899     1
Name: monthly_income, Length: 19150, dtype: int64

Никаких критичных значений не обнаружено.
### Борьба с пропусками
Вспомним, что у нас все еще имеются пропуски в столбике **"monthly_income"**.
Создадим сводную таблицу, где в каждую клетку, соответствующую строке возроста и столбцу семейного положения запишем среднее значение зарплаты.

In [24]:
df1 = pd.pivot_table(df,
               index = ["age"],
               columns = ["education"],
               values = ["monthly_income"],
               aggfunc = [np.mean])
df1.head()

mean                                                 
          monthly_income                                                 
education             вы             на             не             ср  уч
age                                                                      
19                   NaN            NaN  116035.643521  100072.452124 NaN
20          94573.610191            NaN   94965.240054  139693.481826 NaN
21         182077.415699  111133.535135  152814.134129  126526.820367 NaN
22         162960.967159  169493.899228  133393.427201  133084.363996 NaN
23         166909.917031  168908.328372  139644.291258  127452.960629 NaN

Переименуем столбцы полученой тоблицы:

In [25]:
df1.columns = ["ср", "вы", "не", "на", "уч"]

С помощью сводной таблицы заполним пропуски основной таблицы:

In [26]:
def f(x: int, a: int, e: str) -> int:
    """
    Функция принимает на вход значения трех ячеек из столбцов 'monthly_income", 'age', 'education'
    и если ячейка столбцо 'monthly_income' пустая, возвращает значение из сводной таблице, которое 
    соответствует возрасту и уровню образования пустой клетки, если эти данные есть в сводной таблице.
    Если таких даннх нет, то она возвращает среднее значение по всему столбцу 'monthly_income'. Если
    же ячейка изначально не пуста, то функция возвращает ее.
    """
    if pd.isnull(x):
        if pd.isnull(df1.loc[a, e]):
            return everage
        else:
            return df1.loc[a, e]
    else:
        return x
everage = df["monthly_income"].mean()
df["monthly_income"] = df.apply(lambda x: f(x.monthly_income, x.age, x.education), axis=1)

Проверим пропуски, чтобы убедиться, что функция отработала корректно

In [27]:
df.isna().mean()

children               0.0
age                    0.0
education              0.0
family_status          0.0
debt                   0.0
monthly_income         0.0
purpose_of_the_loan    0.0
dtype: float64

Пропусков нет. Предобработка закончена.
### Микро-вывод: предобработка
Предобработка данных является основной и наиболее трудоемкой работой, ведь от того, как она будет проведена, будут ли учтены все моменты и нюансы или нет, зависит качество дальнейшего анализа данных. Поэтому так важно уделить этому этапу достаточное количество времени.
В процессе предобработки мной были удалены неявные и явные дубликаты, также все пропуски были заменены используя и обрабатывая аналогичные данные. Проверка аномальных значений была также проведена и все значения, которые я посчитала отклоненными от нормы, были удалены.
<br>
## Анализ данных
### Выяснить как количество детей у клиента влияет на погашение кредита без задолженностей
Создадим сводную таблицу, где будет количесвто значений столбца 'debt' и сумма по этому столбцу (количество задолжностей) в соответствии с количеством детей:

In [28]:
df2 = pd.pivot_table(df,
               index = ["children"],
               values = ["debt"],
               aggfunc = [np.sum,len])
df2.head(10)

,sum,len
,debt,debt
children,,
0,1058,13926
1,440,4775
2,194,2031
3,27,328
4,4,41
5,0,9


Переименуем столбцы, чтобы было удобнее анализировать оплученные данные.

In [29]:
df2.columns = ["with_debt", "all_debts"]

Создадим еще один столбец **"without_debt"**, в котором запишем значение количества людей без задолжности. Его можно получить вычев из значения столбца **"all"** значение из столбца **"with_debt"**

In [30]:
df2["without_debt"] = df2['all_debts'] - df2['with_debt']
df2.head(10)

,with_debt,all_debts,without_debt
children,,,
0,1058,13926,12868
1,440,4775,4335
2,194,2031,1837
3,27,328,301
4,4,41,37
5,0,9,9


Для того, чтобы удобнее было сделать вывод, посчитаем для каждого количества детей сколько процентов от всех кредитов были без задолжностей:

In [31]:
lst = df2.index
def percent1(w: int, a: int) -> int:
    """ 
    Функция принимает 2 числа для каждого количества детей в семье: количество людей, закрывшых
    кредит без долгов и общее количество людей, взявших кредит. Возвращает процентное соотношение этих значений
    """
    return w * 100/ a
    
    
df2["answer"] = df2.apply(lambda x: percent1(x.without_debt, x.all_debts), axis=1)
df2.head(10)

,with_debt,all_debts,without_debt,answer
children,,,,
0,1058,13926,12868,92.402700
1,440,4775,4335,90.785340
2,194,2031,1837,90.448055
3,27,328,301,91.768293
4,4,41,37,90.243902
5,0,9,9,100.000000


Заметим, что проценты не сильно отличаются. Интересно отметить, что люди, имеющие 5 детей в 100% случаев закрыли кредит без задолжностей, хотя нельзя сказать, что это объективная оценка, т.к. из всего 9 человек.
Остальные же проценты колеблятся от 90 до 92, причем из них наивысший процент у людей без детей (92.4%), а наименьший у людей с 4-мя детьми (90.2%).
<br>
### Выяснить как цель получения кредита влияет на погашение кредита без задолженностей
<br>
Действия не будут отличаться от тех, которые мы проводили ранее. Опять нам понадобитсья свобная таблица, на этот раз вместо количества детей будет выступать показатель причины взятия кредита (столбец **purpose_of_the_loan**). Столбец, отражающий количество людей, закрывших кредит без задолжностей мы получим тем же способом, что и раньше:

In [32]:
df3 = pd.pivot_table(df,
               index = ["purpose_of_the_loan"],
               values = ["debt"],
               aggfunc = [np.sum,len])
df3.columns = ["with_debt", "all_debts"]
df3["without_debt"] = df3['all_debts'] - df3['with_debt']
df3.head(10)

,with_debt,all_debts,without_debt
purpose_of_the_loan,,,
автомобиль,397,4246,3849
жилье,312,4384,4072
недвижимость,182,2516,2334
образование,287,3055,2768
ремонт,222,2780,2558
свадьба,180,2275,2095
строительство,143,1854,1711


In [33]:
lst = df3.index
def percent2(w: int, a: int) -> int:
    """ 
    Функция принимает 2 числа для каждой из причин взятия кредита: количество людей, закрывшых
    его без долгов и общее количество людей, взявших кредит. Возвращает процентное соотношение этих значений
    """
    return w * 100/ a
    
    
df3["answer"] = df3.apply(lambda x: percent2(x.without_debt, x.all_debts), axis=1)
df3.head(10)

,with_debt,all_debts,without_debt,answer
purpose_of_the_loan,,,,
автомобиль,397,4246,3849,90.650024
жилье,312,4384,4072,92.883212
недвижимость,182,2516,2334,92.766296
образование,287,3055,2768,90.605565
ремонт,222,2780,2558,92.014388
свадьба,180,2275,2095,92.087912
строительство,143,1854,1711,92.286947


Проценты погашенных без задолженностей кредитов снова большие (что для меня удивительно), причем диапазон их колебания опять от 90 до 93. Наибольший процент погашения кредитов без задолженностей возникает, когда причина взятия кредита - жилье (92.9%), а наименьший при взятии кредита на образование (90.6%)
<br>
### Выяснить как семейное положение влияет на погашение кредита без задолженностей
<br>
Тут я буду действовать так же. Только в качестве значений названия строк будет выступать столбец **"family_status"**

In [34]:
df4 = pd.pivot_table(df,
               index = ["family_status"],
               values = ["debt"],
               aggfunc = [np.sum,len])
df4.columns = ["with_debt", "all_debts"]
df4["without_debt"] = df4['all_debts'] - df4['with_debt']
df4.head(10)

,with_debt,all_debts,without_debt
family_status,,,
вд,62,943,881
гб,382,4088,3706
ж/з,923,12123,11200
нет,272,2778,2506
р,84,1178,1094


In [35]:
lst = df4.index
def percent3(w: int, a: int) -> int:
    """ 
    Функция принимает 2 числа для каждого из возможных семейных статусов: количество людей, закрывшых
    кредит без долгов и общее количество людей, которые его взяли. Возвращает процентное соотношение 
    этих значений
    """
    return w * 100/ a
    
    
df4["answer"] = df4.apply(lambda x: percent3(x.without_debt, x.all_debts), axis=1)
df4.head(10)



,with_debt,all_debts,without_debt,answer
family_status,,,,
вд,62,943,881,93.425239
гб,382,4088,3706,90.655577
ж/з,923,12123,11200,92.386373
нет,272,2778,2506,90.208783
р,84,1178,1094,92.869270


По статистике видно, что колебание происходит от 90 до 93.4 процентов. Чаще всего отдают кредит без задолжностей вдовы/вдовцы (93.4%), а реже люди не бывшие в браке (90.2%).
<br>
### Микро-вывод: анализ
Анализ данных в целом был достаточно простым за счет своей схожести при решении поставленных задач. Все решалось с помощью составления сводных таблиц и дальнейшей их обработки. Итоговые проценты при ответе на все три вопроса показались мне достаточно странными. Хотя общий процент людей с задолжностями равен всего 8.9 процентов, так что можно понять, почему так получилось
## Вывод
Итак, в процессе моей работы, я провела обзор, предобработку и анализ данных, ответив на все 3 поставленных задачи. Обзор данных был классическим, там присутствовала проверка типов данных, общее количество столбцов и строк и т.п. Предобработка включала в себя замену пропусков не просто на среднее значение, а на среднее значение по категории (возраст и образование). Также в предобработке я поработала с неявными дубликатами. Сам анализ был разделен на задачи. В основном я использовала сводные таблицы для их решения. С их помощью я выбирала нужные значения из всех имеющихся, и соспоставляла с выбранным параметром.Еще я активно использовала метод "apply". Таким образом мне удалось проследить закономерность между количесвтом людей, погасивших кредит без задолжности и семейнм положением/целью взятия кредита/количеством детей.


